<a href="https://colab.research.google.com/github/LevWilliams/ContradictoryMyDearWatson/blob/master/PNClassification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Modules and Libraries

In [2]:
## importing packages
import gc
import os
import random
!pip install transformers
import transformers
import warnings

import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow.keras.backend as K

from pathlib import Path
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold,StratifiedKFold
from tensorflow.keras import Model
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.optimizers import Adam
from transformers import AutoTokenizer, TFAutoModel

print(f"TensorFlow version: {tf.__version__}")
print(f"Transformers version: {transformers.__version__}")

warnings.filterwarnings("ignore")

     |████████████████████████████████| 890kB 3.5MB/s 
     |████████████████████████████████| 890kB 15.5MB/s 
     |████████████████████████████████| 3.0MB 25.9MB/s 
     |████████████████████████████████| 1.1MB 46.2MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=1e92d6ebc21587617ce39b463dc8aea1524bef7ca966e3f923b30c530558bea1
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses
TensorFlow version: 2.3.0
Transformers version: 3.1.0


In [3]:
#tpu
tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
tf.config.experimental_connect_to_cluster(tpu)
tf.tpu.experimental.initialize_tpu_system(tpu)

# instantiate a distribution strategy
tpu_strategy = tf.distribute.experimental.TPUStrategy(tpu)

INFO:absl:Entering into master device scope: /job:worker/replica:0/task:0/device:CPU:0


INFO:tensorflow:Initializing the TPU system: grpc://10.121.20.10:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.121.20.10:8470


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Clearing out eager caches


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:7, TPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


Uploading Data

In [4]:
PolySocMed = pd.read_csv("political_social_media.csv", encoding = 'iso-8859-1')

In [5]:
PolySocMed.head()

,_unit_id,_golden,_unit_state,_trusted_judgments,_last_judgment_at,audience,audience:confidence,bias,bias:confidence,message,message:confidence,orig__golden,audience_gold,bias_gold,bioid,embed,id,label,message_gold,source,text
0,766192484,False,finalized,1,8/4/15 21:17,national,1.0,partisan,1.0,policy,1.0,NaN,NaN,NaN,R000596,"<blockquote class=""twitter-tweet"" width=""450"">...",3.83249E+17,From: Trey Radel (Representative from Florida),NaN,twitter,RT @nowthisnews: Rep. Trey Radel (R- #FL) slam...
1,766192485,False,finalized,1,8/4/15 21:20,national,1.0,partisan,1.0,attack,1.0,NaN,NaN,NaN,M000355,"<blockquote class=""twitter-tweet"" width=""450"">...",3.11208E+17,From: Mitch McConnell (Senator from Kentucky),NaN,twitter,VIDEO - #Obamacare: Full of Higher Costs and ...
2,766192486,False,finalized,1,8/4/15 21:14,national,1.0,neutral,1.0,support,1.0,NaN,NaN,NaN,S001180,"<blockquote class=""twitter-tweet"" width=""450"">...",3.39069E+17,From: Kurt Schrader (Representative from Oregon),NaN,twitter,Please join me today in remembering our fallen...
3,766192487,False,finalized,1,8/4/15 21:08,national,1.0,neutral,1.0,policy,1.0,NaN,NaN,NaN,C000880,"<blockquote class=""twitter-tweet"" width=""450"">...",2.98528E+17,From: Michael Crapo (Senator from Idaho),NaN,twitter,RT @SenatorLeahy: 1st step toward Senate debat...
4,766192488,False,finalized,1,8/4/15 21:26,national,1.0,partisan,1.0,policy,1.0,NaN,NaN,NaN,U000038,"<blockquote class=""twitter-tweet"" width=""450"">...",4.07643E+17,From: Mark Udall (Senator from Colorado),NaN,twitter,.@amazon delivery #drones show need to update ...


In [6]:
TrainAndValDat = PolySocMed[['text','bias']]
TrainAndValDat.head()

,text,bias
0,RT @nowthisnews: Rep. Trey Radel (R- #FL) slam...,partisan
1,VIDEO - #Obamacare: Full of Higher Costs and ...,partisan
2,Please join me today in remembering our fallen...,neutral
3,RT @SenatorLeahy: 1st step toward Senate debat...,neutral
4,.@amazon delivery #drones show need to update ...,partisan


In [7]:
TrainAndValDat['bias']

0       partisan
1       partisan
2        neutral
3        neutral
4       partisan
          ...   
4995    partisan
4996    partisan
4997     neutral
4998     neutral
4999     neutral
Name: bias, Length: 5000, dtype: object

Tokenizing the data

In [8]:
from transformers import AutoTokenizer, AutoModelWithLMHead

In [9]:
#returns tokenized data
def tokenize_sentences(sentences, tokenizer, max_seq_len = 100):
    tokenized_sentences = []
    for sentence in sentences:
        tokenized_sentence = tokenizer.encode(sentence, add_special_tokens = True, max_length = max_seq_len,truncation = True)
        tokenized_sentences.append(tokenized_sentence)
    return tokenized_sentences

def preprocessPoliticalData():
    print("Loading data...")
    x_text = TrainAndValDat['text']
    y = TrainAndValDat['bias']
    tokenizer =  AutoTokenizer.from_pretrained("bert-base-uncased", do_lowercase=True)

    ydict = {
        'neutral':0,
        'partisan':1
    }

    x_train = tokenize_sentences(x_text, tokenizer)
    x = np.array(list(tf.keras.preprocessing.sequence.pad_sequences(x_train, 100, padding='post', truncating='post')))

    shuffle_indices = np.random.permutation(np.arange(len(y)))
    x_train = x[shuffle_indices]
    y_train = y[shuffle_indices].apply(lambda x: ydict[x])

    del x, y
    return x_train, y_train

In [10]:

x_train, y_train = preprocessPoliticalData()

Loading data...


In [11]:
y_train

646     0
3232    1
1839    0
4198    0
1521    1
       ..
643     1
4354    1
3244    1
3224    0
3128    0
Name: bias, Length: 5000, dtype: int64

In [12]:

def get_tf_dataset(X, y, auto, labelled = True, repeat = False, shuffle = False, batch_size = 100):
    """
    Creating tf.data.Dataset for TPU
    """
    if labelled:
        ds = (tf.data.Dataset.from_tensor_slices((X, y)))
    else:
        ds = (tf.data.Dataset.from_tensor_slices(X))

    if repeat:
        ds = ds.repeat()

    if shuffle:
        ds = ds.shuffle(256)

    ds = ds.batch(batch_size)
    ds = ds.prefetch(auto)

    return ds

Model

In [13]:
## building model
def build_model(model_name, max_len):
    """
    Building the Deep Learning architecture
    """
    # defining encoded inputs
    input_ids = Input(shape = (max_len,), dtype = tf.int32, name = "input_ids")
    
    # defining transformer model embeddings
    transformer_model = TFAutoModel.from_pretrained(model_name)
    transformer_embeddings = transformer_model(input_ids)[0]
    
    # defining output layer
    output_values = Dense(2, activation = "softmax")(transformer_embeddings[:, 0, :])

    # defining model
    model = Model(inputs = input_ids, outputs = output_values)
    opt = Adam(learning_rate = 1e-5)
    loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits = True)

    model.compile(optimizer = opt, loss = loss, metrics = [tf.keras.metrics.SparseCategoricalAccuracy()])

    return model

In [14]:
def run_model(xdat,ydat,foldsplits):
      # K-fold on comment and label
    kf = KFold(n_splits = foldsplits, shuffle = True, random_state = 2)
        # initializing predictions
    preds_oof = np.zeros((x_train.shape[0], 3))
    acc_oof = []


    for (fold, (train_index, valid_index)) in enumerate(kf.split(x_train, y_train)):
        # initializing TPU
        K.clear_session()
        with tpu_strategy.scope():
            model = build_model("bert-base-uncased", 100)
            if fold == 0:
                print(model.summary())
                print("\n")
        
        print("#" * 19)
        print(f"##### Fold: {fold + 1} #####")
        print("#" * 19)

        # splitting data into training and validation
        X_train = xdat[train_index]
        X_valid = xdat[valid_index]

        Y_train = ydat[train_index]
        Y_valid = ydat[valid_index]

        # creating TF Dataset
        ds_train = get_tf_dataset(X_train, Y_train, tf.data.experimental.AUTOTUNE, repeat = True, shuffle = True, batch_size = 64)
        ds_valid = get_tf_dataset(X_valid, Y_valid, tf.data.experimental.AUTOTUNE, batch_size = 64)

        n_train = X_train.shape[0]

        # saving model at best accuracy epoch
        sv = tf.keras.callbacks.ModelCheckpoint(
            "model.h5",
            monitor = "val_sparse_categorical_accuracy",
            verbose = 0,
            save_best_only = True,
            save_weights_only = True,
            mode = "max",
            save_freq = "epoch"
        )

        # training model
        model_history = model.fit(
            ds_train,
            epochs = 10,
            callbacks = [sv],
            steps_per_epoch = 100,
            validation_data = ds_valid
        )

        print("\nValidating")

        # scoring validation data
        #model.load_weights("model.h5")
        ds_valid = get_tf_dataset(X_valid, -1, tf.data.experimental.AUTOTUNE, labelled = False, batch_size = 64)

        preds_valid = model.predict(ds_valid)
        acc = accuracy_score(Y_valid, np.argmax(preds_valid, axis = 1))

        #preds_oof[valid_index] = preds_valid
        acc_oof.append(acc)

        print("\nInferencing")

    # overall CV score and standard deviation
    print(f"\nCV Mean Accuracy: {round(np.mean(acc_oof), 4)}")
    print(f"CV StdDev Accuracy: {round(np.std(acc_oof), 4)}\n")

    return np.mean(acc_oof)


In [ ]:
meanmodelacc = run_model(x_train,y_train,5)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the model checkpoint at bert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_ids (InputLayer)       [(None, 100)]             0         
_________________________________________________________________
tf_bert_model (TFBertModel)  ((None, 100, 768), (None, 109482240 
_________________________________________________________________
tf_op_layer_strided_slice (T [(None, 768)]             0         
_________________________________________________________________
dense (Dense)                (None, 2)                 1538      
Total params: 109,483,778
Trainable params: 109,483,778
Non-trainable params: 0
_________________________________________________________________
None


###################
##### Fold: 1 #####
###################
Epoch 1/10
Instructions for updating:
Use `tf.data.Iterator.get_next_as_optional()` instead.


Instructions for updating:
Use `tf.data.Iterator.get_next_as_optional()` instead.


  2/100 [..............................] - ETA: 26:57 - loss: 0.6987 - sparse_categorical_accuracy: 0.5000WARNING:tensorflow:Callbacks method `on_train_batch_end` is slow compared to the batch time (batch time: 0.0078s vs `on_train_batch_end` time: 0.0828s). Check your callbacks.


100/100 [==============================] - ETA: 0s - loss: 0.5970 - sparse_categorical_accuracy: 0.7141WARNING:tensorflow:Callbacks method `on_test_batch_end` is slow compared to the batch time (batch time: 0.0033s vs `on_test_batch_end` time: 0.0251s). Check your callbacks.


...

Some weights of the model checkpoint at bert-base-uncased were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the model checkpoint at bert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


###################
##### Fold: 4 #####
###################
Epoch 1/10


  2/100 [..............................] - ETA: 29:12 - loss: 0.6894 - sparse_categorical_accuracy: 0.6250WARNING:tensorflow:Callbacks method `on_train_batch_end` is slow compared to the batch time (batch time: 0.0072s vs `on_train_batch_end` time: 0.0840s). Check your callbacks.


100/100 [==============================] - ETA: 0s - loss: 0.6011 - sparse_categorical_accuracy: 0.7125WARNING:tensorflow:Callbacks method `on_test_batch_end` is slow compared to the batch time (batch time: 0.0034s vs `on_test_batch_end` time: 0.0243s). Check your callbacks.


100/100 [==============================] - 60s 597ms/step - loss: 0.6011 - sparse_categorical_accuracy: 0.7125 - val_loss: 0.5563 - val_sparse_categorical_accuracy: 0.7570
Epoch 2/10
100/100 [==============================] - 10s 100ms/step - loss: 0.5842 - sparse_categorical_accuracy: 0.7291 - val_loss: 0.5563 - val_sparse_categorical_accuracy: 0.7570
Epoch 3/10
100/100 [==============================] - 10s 101ms/step - loss: 0.5820 - sparse_categorical_accuracy: 0.7312 - val_loss: 0.5563 - val_sparse_categorical_accuracy: 0.7570
Epoch 4/10
100/100 [==============================] - 10s 102ms/step - loss: 0.5711 - sparse_categorical_accuracy: 0.7422 - val_loss: 0.5563 - val_sparse_categorical_accuracy: 0.7570
Epoch 5/10
100/100 [==============================] - 10s 101ms/step - loss: 0.5633 - sparse_categorical_accuracy: 0.7500 - val_loss: 0.5563 - val_sparse_categorical_accuracy: 0.7570
Epoch 6/10
100/100 [==============================] - 10s 102ms/step - loss: 0.6008 - sparse_cat

In [15]:
meanmodelacc

nan